In [1]:
using Revise
using ConvexHullPricing
using LinearAlgebra
using Statistics
using DataFrames
using Plots, LaTeXStrings
using JLD2
using Plots.PlotMeasures
using ProgressBars, JuMP, Gurobi

function mon(array)
  result = Float64[]
  last = array[1]
  for elt in array
    new_last = minimum([elt, last])
    push!(result, new_last)
    last = new_last
  end
  return result
end

PER_RANGE = 4:5:100
const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer

instances = []
for file in readdir("..//data//ca"; join=true)
  push!(instances, UT.load_data(file))
end

function getavglast10(array)
  percentage = 10/100
  last_index = length(array) - Int(ceil(length(array) * percentage))
  return mean(array[last_index:end])
end

function get_last_elements(array)
  result = []
  for i in PER_RANGE
    percentage = i / 100
    last_index = length(array) - Int(ceil(length(array) * percentage))
    last_elements = mean(array[last_index:end])
    push!(result, last_elements)
  end
  return result
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01


get_last_elements (generic function with 1 method)

In [2]:
function best_indices(f_array)
  biggest_so_far = f_array[1]
  indices = []
  for (i, elt) in enumerate(f_array)
    if elt > biggest_so_far
      push!(indices, i)
      biggest_so_far = elt
    end
  end
  return indices
end

function best_indices_plateau(f_array)
  biggest_so_far = f_array[1]
  indices = []
  last_index = 1
  for (i, elt) in enumerate(f_array)
    if elt > biggest_so_far
      last_index = i
      biggest_so_far = elt
    end
    push!(indices, last_index)
  end
  return indices
end

best_indices_plateau (generic function with 1 method)

In [ ]:
FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")

In [ ]:
methods = ["BundleLevelMethodBE", "BundleProximalLevelMethod-L-BE", "D-AdaptationBE", "DowGBE", "SUBG-EP-BE", "SubGBE"]
methods_name = ["Bundle Level Method", "Bundle Proximal Level Method", "D-Adaptation", "DowG", "Estimated Polyak stepsizes, subgradient method", "Subgradient method"]

In [ ]:
methods_name = ["BLM", "BPLM", "DA", "DowG", "EST-Pol", "Subg"]

In [ ]:
arr_diff = []
for (j, meth) in tqdm(enumerate(methods))
  all_dists = []
  for idx in tqdm(1:8)
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")
      avg_it_val = FS - UT.exact_oracle(instance, getavglast10(ITBLM))[1]
      best_it_val = minimum(mon(FS .- (BLM)))
      push!(all_dists, best_it_val - avg_it_val)
  end
  push!(arr_diff, mean(all_dists))
end
bar(arr_diff, xticks=(1:6, methods_name), legend=false, ylabel="Difference between best and average", title="Difference between best and average for each method", size=(800, 400), fmt=:png)

In [ ]:
arr = []
for (j, meth) in tqdm(enumerate(methods))
  all_dists = []
  for idx in tqdm(1:8)
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")
      best_it_val = minimum(mon(FS .- (BLM)))
      push!(all_dists, best_it_val)
  end
  push!(arr, mean(all_dists))
end
bar(arr, xticks=(1:6, methods_name), legend=false, ylabel="Difference between best and average", title="Difference between best and average for each method", size=(800, 400), fmt=:png)


In [ ]:
using StatsPlots

In [ ]:
M = rand(6,2)
M[:,1] = arr
M[:,2] = arr .- arr_diff

In [ ]:
plot(xlabel = "Optimization methods",
  ylabel = L"\mathcal{L}_* - \mathcal{L}",
  title="Average on all belgian instances",
  titlefontsize=10,
  legend=:topright,
  labelfontsize=10,
  size=(1000,500),
  bottom_margin = 15px,
  left_margin = 30px,
  bar_position = :dodge,
  fmt=:pdf)
groupedbar!(M, bar_position = :dodge, bar_width=0.5, bar_gap=0, xticks=(1:6, methods_name), label = ["Best iterate" "10% average iterate"], legend=:topright)
savefig("AverageItBE.pdf")

In [ ]:
for (j, meth) in enumerate(methods)
  all_values = []
  all_distances = []
  for idx=1:20
    if idx ∉ [9 10 11 12]
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunCA$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")
      average_iterates = get_last_elements(ITBLM)
      values = []
      for avg in average_iterates
        push!(values, FS - UT.exact_oracle(instance, avg)[1])
      end
      push!(all_values, values)
      plot(
        title = "Californian instance $(idx)",
        xlabel = "Percentage of last iterations considered " * " (%)",
        ylabel = L"\mathcal{L}_* - \mathcal{L}",
        xlims=(0,100),
        xticks=5:5:100,
        titlefontsize=10,
        legend=:topright,
        labelfontsize=10,
        size=(1000,500),
        bottom_margin = 15px,
        left_margin = 30px,
        fmt=:pdf
      )
      plot!(5:5:100, values, label="$(methods_name[j])")
      hline!([minimum(mon(FS .- (BLM)))], label = "Best iterate")
      push!(all_distances, minimum(mon(FS .- (BLM))))
      savefig("average_iterate5_100/$(meth)$(idx).pdf")
    end
  end
  plot(
    title = "Belgian dataset",
    xlabel = "Percentage of last iterations considered " * " (%)",
    ylabel = L"\mathcal{L}_* - \mathcal{L}",
    xlims=(0,100),
    xticks=5:5:100,
    titlefontsize=10,
    legend=:topright,
    labelfontsize=10,
    size=(1000,500),
    bottom_margin = 15px,
    left_margin = 30px,
    fmt=:pdf
  )
  plot!(5:5:100, mean(all_values), label="$(methods_name[j])")
  hline!([mean(all_distances)], label = "Best iterate")
  savefig("average_iterate5_100/$(meth).pdf")
end

In [ ]:
# Version with iterates that give decrease only:
for (j, meth) in enumerate(methods)
  all_values = []
  all_distances = []
  for idx=1:20
    if idx ∉ [9, 10, 11, 12]
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunCA$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")
      ITBLM = ITBLM[2:end][best_indices_plateau(BLM)]
      average_iterates = get_last_elements(ITBLM)
      values = []
      for avg in average_iterates
        push!(values, FS - UT.exact_oracle(instance, avg)[1])
      end
      push!(all_values, values)
      plot(
        title = "Californian instance $(idx)",
        xlabel = "Percentage of last iterations considered " * " (%)",
        ylabel = L"\mathcal{L}_* - \mathcal{L}",
        xlims=(PER_RANGE[1],PER_RANGE[end]),
        xticks=PER_RANGE,
        titlefontsize=10,
        legend=:topright,
        labelfontsize=10,
        size=(1000,500),
        bottom_margin = 15px,
        left_margin = 30px,
        fmt=:pdf
      )
      plot!(PER_RANGE, values, label="$(methods_name[j])")
      hline!([minimum(mon(FS .- (BLM)))], label = "Best iterate")
      push!(all_distances, minimum(mon(FS .- (BLM))))
      savefig("avg_best_plateau/CA dataset/$(meth)$(idx).pdf")
    end
  end
  plot(
    title = "Californian dataset",
    xlabel = "Percentage of last iterations considered " * " (%)",
    ylabel = L"\mathcal{L}_* - \mathcal{L}",
    xlims=(PER_RANGE[1],PER_RANGE[end]),
    xticks=PER_RANGE,
    titlefontsize=10,
    legend=:topright,
    labelfontsize=10,
    size=(1000,500),
    bottom_margin = 15px,
    left_margin = 30px,
    fmt=:pdf
  )
  plot!(PER_RANGE, mean(all_values), label="$(methods_name[j])")
  hline!([mean(all_distances)], label = "Best iterate")
  savefig("avg_best_plateau/CA dataset/$(meth).pdf")
end

In [ ]:
instances = []
for file in readdir("..//data//belgian"; join=true)
  push!(instances, UT.load_data(file))
end

methods = ["BundleLevelMethodBE", "BundleProximalLevelMethod-L-BE", "D-AdaptationBE", "DowGBE", "SUBG-EP-BE", "SubGBE"]
methods_name = ["Bundle Level Method", "Bundle Proximal Level Method", "D-Adaptation", "DowG", "Estimated Polyak stepsizes, subgradient method", "Subgradient method"]

In [ ]:
# Version with iterates that give decrease only:
for (j, meth) in enumerate(methods)
  all_values = []
  all_distances = []
  for idx=1:8
      instance = instances[idx]
      FS = maximum(load_object("..//results//optimal_values//NewRefinedOptRunBE$(idx).jld2")[3])
      XBLM, ITBLM, BLM, TBLM = load_object("..//results//15min_runs//$(meth)$(idx).jld2")
      ITBLM = ITBLM[2:end][best_indices_plateau(BLM)]
      average_iterates = get_last_elements(ITBLM)
      values = []
      for avg in average_iterates
        push!(values, FS - UT.exact_oracle(instance, avg)[1])
      end
      push!(all_values, values)
      plot(
        title = "Belgian instance $(idx)",
        xlabel = "Percentage of last iterations considered " * " (%)",
        ylabel = L"\mathcal{L}_* - \mathcal{L}",
        xlims=(PER_RANGE[1],PER_RANGE[end]),
        xticks=PER_RANGE,
        titlefontsize=10,
        legend=:topright,
        labelfontsize=10,
        size=(1000,500),
        bottom_margin = 15px,
        left_margin = 30px,
        fmt=:pdf
      )
      plot!(PER_RANGE, values, label="$(methods_name[j])")
      hline!([minimum(mon(FS .- (BLM)))], label = "Best iterate")
      push!(all_distances, minimum(mon(FS .- (BLM))))
      savefig("avg_best_plateau/BE dataset/$(meth)$(idx).pdf")
  end
  plot(
    title = "Belgian dataset",
    xlabel = "Percentage of last iterations considered " * " (%)",
    ylabel = L"\mathcal{L}_* - \mathcal{L}",
    xlims=(PER_RANGE[1],PER_RANGE[end]),
    xticks=PER_RANGE,
    titlefontsize=10,
    legend=:topright,
    labelfontsize=10,
    size=(1000,500),
    bottom_margin = 15px,
    left_margin = 30px,
    fmt=:pdf
  )
  plot!(PER_RANGE, mean(all_values), label="$(methods_name[j])")
  hline!([mean(all_distances)], label = "Best iterate")
  savefig("avg_best_plateau/BE dataset/$(meth).pdf")
end